In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Load submission
Sub = pd.read_csv('../input/2021-ncaaw-first-step-v-3-visualization-result/submission.csv')

In [ ]:
Teams = pd.read_csv('../input/ncaaw-march-mania-2021/WDataFiles_Stage2/WTeams.csv')
Slots = pd.read_csv('../input/ncaaw-march-mania-2021/WDataFiles_Stage2/WNCAATourneySlots.csv')
Seeds = pd.read_csv('../input/ncaaw-march-mania-2021/WDataFiles_Stage2/WNCAATourneySeeds.csv')
Seeds = Seeds.merge(Teams,how = 'left',left_on = ['TeamID'],right_on = ['TeamID'])
Sub['ATeamID'] = Sub['ID'].apply(lambda x: int(x.split('_')[1]))
Sub['BTeamID'] = Sub['ID'].apply(lambda x: int(x.split('_')[2]))
Sub = Sub[['ATeamID','BTeamID','Pred']]
Sub = Sub.merge(Seeds[Seeds['Season']==2021],how= 'left',left_on = 'ATeamID',right_on = 'TeamID').drop(['TeamID','Season'], axis=1)
Sub = Sub.merge(Seeds[Seeds['Season']==2021],how= 'left',left_on = 'BTeamID',right_on = 'TeamID').drop(['TeamID'], axis=1).rename(columns={'Seed_x': 'ASeed','Seed_y': 'BSeed','TeamName_x':'ATeamName','TeamName_y':'BTeamName'})
Sub = Sub.drop(['Season','ATeamID','BTeamID'],axis = 1)

FirstRound = Slots[0:32]
SecondRound = Slots[32:48]
Sweet16 = Slots[48:56]
EliteEight = Slots[56:60]
FinalFour = Slots[60:62]
Final = Slots[62:]
FirstRound['ASeed'] = FirstRound['StrongSeed']
FirstRound['BSeed'] = FirstRound['WeakSeed']

def merger(Round,Sub=Sub):  
    Round = Round.merge(Sub,how = 'left',left_on = ['ASeed','BSeed'],right_on = ['ASeed','BSeed'])
    for index in Round[Round['Pred'].isna() == True].index:
            Round.at[index, 'Temp'] = Round.at[index, 'ASeed']
            Round.at[index, 'ASeed'] = Round.at[index, 'BSeed']
            Round.at[index, 'BSeed'] = Round.at[index, 'Temp']
            Round = Round.drop(['Temp'],axis = 1)
            
    Round = Round.merge(Sub,how = 'left',left_on = ['ASeed','BSeed'],right_on = ['ASeed','BSeed'])
    Round = Round.drop(['Pred_x','ATeamName_x','BTeamName_x'],axis = 1).rename(columns={'Pred_y': 'Pred','ATeamName_y':'ATeamName','BTeamName_y':'BTeamName'})
    Round['Win'] = Round['Pred'] > 0.5
    Round[['Win_Seed']] = Round[Round['Win'] == True][['ASeed']]
    Round[['Win_Name']] = Round[Round['Win'] == True][['ATeamName']]
    Round['Win_Seed'].fillna(value=Round[Round['Win'] == False]['BSeed'], inplace=True)
    Round['Win_Name'].fillna(value=Round[Round['Win'] == False]['BTeamName'], inplace=True)
    return Round 

# First Round Predict

In [ ]:
FirstRound = merger(FirstRound)
FirstRound_Win = FirstRound[['Slot','Win_Seed']]
for game in FirstRound.values:
    print('{:4}({:6.2%}){:^16}{:2}{:^16}({:6.2%}){:>4}'.format(game[3],game[5], game[6],'vs', game[7],1-game[5],game[4]))
    print('{:>25} {:<} {:<}'.format('WINNER:',game[10],game[9]))
    print('-'*58)

# Second Round Predict

In [ ]:
SecondRound = SecondRound.merge(FirstRound_Win,how= 'left', left_on = 'StrongSeed',right_on = 'Slot')
SecondRound = SecondRound.merge(FirstRound_Win,how= 'left', left_on = 'WeakSeed',right_on = 'Slot').drop(['Slot','Slot_y'],axis=1).rename(columns={'Win_Seed_x': 'ASeed','Win_Seed_y': 'BSeed','Slot_x':'Slot'})
SecondRound = merger(SecondRound)
SecondRound_Win = SecondRound[['Slot','Win_Seed']]
for game in SecondRound.values:
    print('{:4}({:6.2%}){:^16}{:2}{:^16}({:6.2%}){:>4}'.format(game[3],game[5], game[6],'vs', game[7],1-game[5],game[4]))
    print('{:>25} {:<} {:<}'.format('WINNER:',game[10],game[9]))
    print('-'*58)

# Sweet 16 Predict

In [ ]:
Sweet16 = Sweet16.merge(SecondRound_Win,how= 'left', left_on = 'StrongSeed',right_on = 'Slot')
Sweet16 = Sweet16.merge(SecondRound_Win,how= 'left', left_on = 'WeakSeed',right_on = 'Slot').drop(['Slot','Slot_y'],axis=1).rename(columns={'Win_Seed_x': 'ASeed','Win_Seed_y': 'BSeed','Slot_x':'Slot'})
Sweet16 = merger(Sweet16)
Sweet16_Win = Sweet16[['Slot','Win_Seed']]
for game in Sweet16.values:
    print('{:4}({:6.2%}){:^16}{:2}{:^16}({:6.2%}){:>4}'.format(game[3],game[5], game[6],'vs', game[7],1-game[5],game[4]))
    print('{:>25} {:<} {:<}'.format('WINNER:',game[10],game[9]))
    print('-'*58)

# Elite Eight Predict

In [ ]:
EliteEight = EliteEight.merge(Sweet16_Win,how= 'left', left_on = 'StrongSeed',right_on = 'Slot')
EliteEight = EliteEight.merge(Sweet16_Win,how= 'left', left_on = 'WeakSeed',right_on = 'Slot').drop(['Slot','Slot_y'],axis=1).rename(columns={'Win_Seed_x': 'ASeed','Win_Seed_y': 'BSeed','Slot_x':'Slot'})
EliteEight = merger(EliteEight)
EliteEight_Win = EliteEight[['Slot','Win_Seed']]
for game in EliteEight.values:
    print('{:4}({:6.2%}){:^16}{:2}{:^16}({:6.2%}){:>4}'.format(game[3],game[5], game[6],'vs', game[7],1-game[5],game[4]))
    print('{:>25} {:<} {:<}'.format('WINNER:',game[10],game[9]))
    print('-'*58)

# Final Four Predict

In [ ]:
FinalFour = FinalFour.merge(EliteEight_Win,how= 'left', left_on = 'StrongSeed',right_on = 'Slot')
FinalFour = FinalFour.merge(EliteEight_Win,how= 'left', left_on = 'WeakSeed',right_on = 'Slot').drop(['Slot','Slot_y'],axis=1).rename(columns={'Win_Seed_x': 'ASeed','Win_Seed_y': 'BSeed','Slot_x':'Slot'})
FinalFour = merger(FinalFour)
FinalFour_Win = FinalFour[['Slot','Win_Seed']]
for game in FinalFour.values:
    print('{:4}({:6.2%}){:^16}{:2}{:^16}({:6.2%}){:>4}'.format(game[3],game[5], game[6],'vs', game[7],1-game[5],game[4]))
    print('{:>25} {:<} {:<}'.format('WINNER:',game[10],game[9]))
    print('-'*58)

# Final Predict

In [ ]:
Final = Final.merge(FinalFour_Win,how= 'left', left_on = 'StrongSeed',right_on = 'Slot')
Final = Final.merge(FinalFour_Win,how= 'left', left_on = 'WeakSeed',right_on = 'Slot').drop(['Slot','Slot_y'],axis=1).rename(columns={'Win_Seed_x': 'ASeed','Win_Seed_y': 'BSeed','Slot_x':'Slot'})
Final = merger(Final)
Final_Win = Final[['Slot','Win_Seed']]
for game in Final.values:
    print('{:4}({:6.2%}){:^16}{:2}{:^16}({:6.2%}){:>4}'.format(game[3],game[5], game[6],'vs', game[7],1-game[5],game[4]))
    print('{:>25} {:<} {:<}'.format('WINNER:',game[10],game[9]))
    print('-'*58)